In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from Custom_dataset import CustomDataset

In [2]:
adult_train = pd.read_csv('./dataset/ECG_adult_age_train.csv')
child_train = pd.read_csv('./dataset/ECG_child_age_train.csv')

In [3]:
adult_file_path = './dataset/ECG_adult_numpy_train/'
child_file_path = './dataset/ECG_child_numpy_train/'

In [4]:
adult_train_data, adult_valid_data = train_test_split(adult_train,shuffle=True, random_state=42, test_size=0.1)
adult_train_data.reset_index(inplace=True,drop=True)
adult_valid_data.reset_index(inplace=True,drop=True)

In [5]:
adult_train_dataset = CustomDataset(adult_file_path,adult_train_data)
adult_valid_dataset = CustomDataset(adult_file_path,adult_valid_data)
child_dataset = CustomDataset(child_file_path,child_train)

In [6]:
train_adult_loader = torch.utils.data.DataLoader(adult_train_dataset, batch_size = 64)
valid_adult_loader = torch.utils.data.DataLoader(adult_valid_dataset, batch_size = 64)

In [12]:
class first_block(torch.nn.Module):
    def __init__(self, N, K):
        super().__init__()

        self.conv_layer = torch.nn.LazyConv1d(N, K, stride=1, padding = 'same')
        self.bn = torch.nn.BatchNorm1d(N)
        self.relu = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool1d(2, 2)

    def forward(self, x):
        out = self.conv_layer(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.maxpool(out)
        return out

class stage_block_1(torch.nn.Module):
    def __init__(self, N):
        super().__init__()

        self.conv_layer1 = torch.nn.LazyConv1d(N, 3, 1, padding='same')
        self.bn1 = torch.nn.BatchNorm1d(N)
        self.conv_layer2 = torch.nn.LazyConv1d(N, 3, 1, padding='same')
        self.bn2 = torch.nn.BatchNorm1d(N)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        res = x
        out = self.conv_layer1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv_layer2(out)
        out = self.bn2(out)
        
        out += res
        return out

class stage_block_2(torch.nn.Module):
    def __init__(self, N):
        super().__init__()

        self.conv_layer1 = torch.nn.LazyConv1d(N, 3, 1, padding='same')
        self.bn1 = torch.nn.BatchNorm1d(N)
        self.conv_layer2 = torch.nn.LazyConv1d(N, 3, 1, padding='same')
        self.bn2 = torch.nn.BatchNorm1d(N)
        self.downsample = torch.nn.Sequential(torch.nn.LazyConv1d(N,1,1,padding='same'), torch.nn.BatchNorm1d(N))
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv_layer2(out)
        out = self.bn2(out)
        res = self.downsample(x)

        out += res
        return out

class Custom_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.first_block = first_block(64,7)
        self.layer1 = torch.nn.Sequential(stage_block_1(64),stage_block_1(64))
        self.layer2 = torch.nn.Sequential(stage_block_2(128),stage_block_1(128))
        self.layer3 = torch.nn.Sequential(stage_block_2(256), stage_block_1(256))
        self.layer4 = torch.nn.Sequential(stage_block_2(512), stage_block_1(512))
        self.AdaptiveAvgPool = torch.nn.AdaptiveAvgPool1d(output_size=1)
        self.flatten = torch.nn.Flatten()
        self.fc = torch.nn.Linear(512,1,bias=True)
    def forward(self,x):
        out = self.first_block(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.AdaptiveAvgPool(out)
        out = self.flatten(out)
        out = self.fc(out)
        return out

In [13]:
model = Custom_model()
model.to('cuda')
print('model_ready')

model_ready


c:\Anaconda\envs\colon\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [14]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr = 3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', min_lr = 1e-5, patience=2)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max= 20, eta_min=1e-5)

In [15]:
for epochs in range(3):
    print('train_run')
    for batch in tqdm(train_adult_loader):
        inputs = batch[0].view(-1,12,5000).to('cuda')
        label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
        output = model(torch.tensor(inputs, dtype = torch.float32))
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    valid_loss = 0
    print('valid_run')
    for batch in tqdm(valid_adult_loader):
        inputs = batch[0].view(-1,12,5000).to('cuda')
        label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
        with torch.no_grad():
            output = model(torch.tensor(inputs, dtype = torch.float32))
        loss = criterion(output, label)
        valid_loss += loss
    loss_check = valid_loss/len(valid_adult_loader)
    print(loss_check)
    scheduler.step(loss_check)

train_run


  0%|          | 0/491 [00:00<?, ?it/s]C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_17520\965771277.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_17520\965771277.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs, dtype = torch.float32))
c:\Anaconda\envs\colon\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(inpu

In [13]:
pred = []
labels = []
for batch in tqdm(valid_adult_loader):
    inputs = batch[0].view(-1,12,5000).to('cuda')
    label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
    with torch.no_grad():
        output = model(torch.tensor(inputs, dtype = torch.float32))
    pred += output.detach().cpu().tolist()
    labels += label.detach().cpu().tolist()

  0%|          | 0/55 [00:00<?, ?it/s]C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_10440\628552704.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_10440\628552704.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs, dtype = torch.float32))
100%|██████████| 55/55 [00:03<00:00, 16.12it/s]


In [14]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(pred,labels)

13.215160969778173